# 추천 시스템: 콘텐츠 기반 추천 시스템

In [260]:
import warnings
warnings.filterwarnings(action='ignore')

In [261]:
import pandas as pd
pd.set_option('max_colwidth', 100)

In [ ]:
df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [263]:
moview_df = df[['id', 'title','genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
moview_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


In [264]:
moview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [265]:
moview_df = moview_df.drop(['id'], axis=1)

In [266]:
from ast import literal_eval

result = literal_eval(moview_df.iloc[0]['genres'])
result

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

### 데이터 전처리

##### 장르

In [267]:
moview_df['genres'] = moview_df['genres'].apply(literal_eval)
moview_df['genres']

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2              [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
                                                       ...                                                 
4798            [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                       [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'...
4801                        

In [268]:
import re

name_result = []
for genre_temp in moview_df['genres']:
    temp_name = []
    for temp in genre_temp:
        name = temp['name']
        temp_name.append(re.sub('[\s]', '', name))
    name_result.append(temp_name)
name_result

moview_df['genres'] = name_result

In [269]:
data_result = []
for temp in moview_df['genres']:
    data = ' '.join(temp)
    data_result.append(data)

moview_df['genres_literal'] = data_result

##### 키워드

In [270]:
moview_df['keywords'] = moview_df['keywords'].apply(literal_eval)
moview_df['keywords']

0       [{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...
1       [{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic is...
2       [{'id': 470, 'name': 'spy'}, {'id': 818, 'name': 'based on novel'}, {'id': 4289, 'name': 'secret...
3       [{'id': 849, 'name': 'dc comics'}, {'id': 853, 'name': 'crime fighter'}, {'id': 949, 'name': 'te...
4       [{'id': 818, 'name': 'based on novel'}, {'id': 839, 'name': 'mars'}, {'id': 1456, 'name': 'medal...
                                                       ...                                                 
4798    [{'id': 5616, 'name': 'united states–mexico barrier'}, {'id': 33649, 'name': 'legs'}, {'id': 162...
4799                                                                                                     []
4800    [{'id': 248, 'name': 'date'}, {'id': 699, 'name': 'love at first sight'}, {'id': 2398, 'name': '...
4801                        

In [271]:
name_result = []
for keyword_temp in moview_df['keywords']:
    temp_name = []
    for temp in keyword_temp:
        name = temp['name']
        temp_name.append(name)
    name_result.append(temp_name)
name_result

moview_df['keywords'] = name_result

### 카운트 벡터라이징

In [272]:
moview_df['genres_literal']

0       Action Adventure Fantasy ScienceFiction
1                      Adventure Fantasy Action
2                        Action Adventure Crime
3                   Action Crime Drama Thriller
4               Action Adventure ScienceFiction
                         ...                   
4798                      Action Crime Thriller
4799                             Comedy Romance
4800               Comedy Drama Romance TVMovie
4801                                           
4802                                Documentary
Name: genres_literal, Length: 4803, dtype: object

In [273]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df = 1, ngram_range = (1, 1)) 
# ngram_range 단어를 1개 에서 2개의 조합을 만들고
# min_df 전체 데이터에서 한번 이상 등장한 조합만 사용

genre_vect = count_vect.fit_transform(moview_df['genres_literal'])

In [274]:
count_vect.vocabulary_

{'action': 0,
 'adventure': 1,
 'fantasy': 8,
 'sciencefiction': 15,
 'crime': 4,
 'drama': 6,
 'thriller': 16,
 'animation': 2,
 'family': 7,
 'western': 19,
 'comedy': 3,
 'romance': 14,
 'horror': 11,
 'mystery': 13,
 'history': 10,
 'war': 18,
 'music': 12,
 'documentary': 5,
 'foreign': 9,
 'tvmovie': 17}

In [275]:
genre_vect.shape

(4803, 20)

In [276]:
genre_vect_array = genre_vect.toarray()
genre_vect_array

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 20))

In [277]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_vect_array, genre_vect_array)
genre_sim[0].argsort()[::-1]

array([   0,   14, 3494, ..., 2979, 2977, 4753], shape=(4803,))

In [278]:
moview_df['title'].iloc[0]

'Avatar'

In [279]:
moview_df['title'].iloc[46]

'X-Men: Days of Future Past'

In [280]:
moview_df['title'].iloc[3569]

'Paranormal Activity: The Marked Ones'

In [281]:
def find_sim_movie(df, sorted_index, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values[0]

    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)

    return df.iloc[similar_index]

genre_sim_sorted = genre_sim.argsort()[:, ::-1]

movie_result = find_sim_movie(df, genre_sim_sorted, 'Superman', 10)
print(movie_result['title'])

print(df.iloc[3494]['genres'])
print(df.iloc[813]['genres'])

0                                          Avatar
14                                   Man of Steel
3494    Beastmaster 2: Through the Portal of Time
46                     X-Men: Days of Future Past
61                              Jupiter Ascending
813                                      Superman
232                                 The Wolverine
10                               Superman Returns
870                                   Superman II
1230                                   The Shadow
Name: title, dtype: object
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
